In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import svm

In [3]:
data=pd.read_csv('data/rap_dataset.tsv', sep='\t')
data.head()

,User,PQLevel,Client,Age,Gender,Country,Protocol,TestDate,R,ResponseOrder,...,Determinants,Pair,FormQuality,FQText,Contents,Popular,ZCode,ZScore,SpecialScores,Rejection
0,136,C,31852,28,Male,Israel,32669,4/28/2017 1:06,22,2,...,FC',0,o,NaN,"Hd,Hx",0,ZS,3.5,PHR,0
1,136,C,31852,28,Male,Israel,32669,4/28/2017 1:06,22,1,...,F,1,-,NaN,A,0,NaN,NaN,NaN,0
2,136,C,31852,28,Male,Israel,32669,4/28/2017 1:06,22,3,...,F,1,-,NaN,"Ad,A",0,ZA,3.0,FAB,0
3,136,C,31852,28,Male,Israel,32669,4/28/2017 1:06,22,4,...,C'F,0,-,NaN,Hd,0,ZW,5.5,PHR,0
4,136,C,31852,28,Male,Israel,32669,4/28/2017 1:06,22,5,...,F,1,o,NaN,A,0,NaN,NaN,NaN,0


In [4]:
data.columns

Index(['User', 'PQLevel', 'Client', 'Age', 'Gender', 'Country', 'Protocol',
       'TestDate', 'R', 'ResponseOrder', 'cardID', 'Location',
       'LocationNumber', 'DevelopmentalQuality', 'Determinants', 'Pair',
       'FormQuality', 'FQText', 'Contents', 'Popular', 'ZCode', 'ZScore',
       'SpecialScores', 'Rejection'],
      dtype='object')

In [5]:
df1=data.dropna()
df1

,User,PQLevel,Client,Age,Gender,Country,Protocol,TestDate,R,ResponseOrder,...,Determinants,Pair,FormQuality,FQText,Contents,Popular,ZCode,ZScore,SpecialScores,Rejection
121,272,C,40624,10,Male,Chile,41595,5/22/2018 14:27,15,1,...,Ma,1,u,Animal (Winged but unspecified),A,0,ZW,1.0,"FAB,PHR",0
129,272,C,40624,10,Male,Chile,41595,5/22/2018 14:27,15,9,...,Mp,0,u,Face (Photo negative and includes use of DS7 o...,(H),0,ZW,2.5,GHR,0
130,272,C,40624,10,Male,Chile,41595,5/22/2018 14:27,15,10,...,"Ma,FC",0,o,Flower,Bt,0,ZW,4.5,GHR,0
144,272,C,41287,10,Male,Israel,42274,6/16/2018 14:52,18,9,...,F,0,o,Human-Like Figure,(H),1,ZW,2.0,GHR,0
155,272,C,42937,15,Male,Afghanistan,43935,9/10/2018 6:36,20,2,...,"Mp,Fr",0,o,Humans (Two),H,0,ZA,4.0,GHR,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506392,29930,C,58299,28,Not Specified,United States (USA),59457,9/10/2020 20:25,19,8,...,"Fr,FMp",0,o,Monkey,A,0,ZA,3.0,"COP,GHR",0
506395,29930,C,58299,28,Not Specified,United States (USA),59457,9/10/2020 20:25,19,11,...,FT,0,o,Moth,A,0,ZW,1.0,PER,0
506398,29930,C,58299,28,Not Specified,United States (USA),59457,9/10/2020 20:25,19,14,...,"FV,FMa",1,-,"Shrimp (Two, natural form)","A,Na",0,ZW,2.5,"COP,PHR",0
506451,29942,B,58333,24,Not Specified,Georgia,59491,9/12/2020 15:32,8,7,...,"FD,CF",0,-,Face,(Hd),0,ZW,5.5,"PHR,ALOG",0


In [6]:
text=np.array(df1['FQText']) #we create a NumPy array with the text/answer
labels=np.array(df1['Contents']) #other array for the coding of this answer



In [7]:
x_train,x_test,y_train,y_test=train_test_split(text,labels,test_size=0.01,random_state=1) #Change test_size

tf_vectorizer=TfidfVectorizer(min_df=2,max_df=0.3,ngram_range=(1,3)) #change parameter
x_train_tfidf=tf_vectorizer.fit_transform(x_train)
x_test_tfidf=tf_vectorizer.transform(x_test)

clf = svm.LinearSVC(dual=True, max_iter=1000)
clf.fit(x_train_tfidf, y_train)

clf.score(x_test_tfidf,y_test)

0.5763888888888888

In [24]:
'''a_pred=clf.predict(x_test_tfidf)

from sklearn.metrics import classification_report
print(classification_report(x_test, a_pred))'''

'a_pred=clf.predict(x_test_tfidf)\n\nfrom sklearn.metrics import classification_report\nprint(classification_report(x_test, a_pred))'

In [8]:
def classify(article_text):
    global clf
    global tf_vectorizer
    global data
    article=tf_vectorizer.transform([article_text])
    
    return clf.predict(article)[0]

In [10]:
classify("a man") #change string here

'A'

ts=0.01 random state 1 : 0.576388

ts=0.16 rs 2 : 0.493252

In [6]:
ts=0.01
results=[]

while ts<0.2:
    x_train,x_test,y_train,y_test=train_test_split(text,labels,test_size=ts, random_state=2)

    tf_vectorizer=TfidfVectorizer(min_df=2,max_df=0.3,ngram_range=(1,3))
    x_train_tfidf=tf_vectorizer.fit_transform(x_train)
    x_test_tfidf=tf_vectorizer.transform(x_test)

    clf = svm.LinearSVC(dual=True, max_iter=1000)
    clf.fit(x_train_tfidf, y_train)

    a=clf.score(x_test_tfidf,y_test)
    print(ts,a)  
    results.append(a)  
    ts=ts+0.01

0.01 0.3888888888888889
0.02 0.4166666666666667
0.03 0.4431554524361949
0.04 0.4539130434782609
0.05 0.46657381615598886
0.060000000000000005 0.4709976798143852
0.07 0.4696517412935323
0.08 0.46823324630113144
0.09 0.47832817337461303
0.09999999999999999 0.4714484679665738
0.10999999999999999 0.47181760607979734
0.11999999999999998 0.477655252466628
0.12999999999999998 0.4790996784565916
0.13999999999999999 0.481592039800995
0.15 0.4886205294937297
0.16 0.4932520679146713
0.17 0.4926229508196721
0.18000000000000002 0.49303405572755415
0.19000000000000003 0.49321598826549323


#### Test with determinants

In [13]:
text2=np.array(df1['FQText']) #we create a NumPy array with the text/answer
labels2=np.array(df1['Determinants']) #other array for the coding of this answer

x_train,x_test,y_train,y_test=train_test_split(text2,labels2,test_size=0.01,random_state=1) #Change test_size

tf_vectorizer=TfidfVectorizer(min_df=2,max_df=0.3,ngram_range=(1,3)) #change parameter
x_train_tfidf=tf_vectorizer.fit_transform(x_train)
x_test_tfidf=tf_vectorizer.transform(x_test)

clf2 = svm.LinearSVC(dual=True, max_iter=1000)
clf2.fit(x_train_tfidf, y_train)

clf2.score(x_test_tfidf,y_test)

0.4444444444444444

In [14]:
def classify2(article_text):
    global clf2
    global tf_vectorizer
    global data
    article=tf_vectorizer.transform([article_text])
    
    return clf2.predict(article)[0]

In [18]:
classify2("A dog")

'FMa'

#### RandomForest Classifier try (with content)

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

data=pd.read_csv('data/rap_dataset.tsv', sep='\t')
df1=data.dropna()


X_train, X_test, y_train, y_test = train_test_split(df1['FQText'], df1['Contents'], test_size=0.2, random_state=42)


tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Réglez max_features selon vos besoins
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)

y_pred = rf_classifier.predict(X_test_tfidf)

print(classification_report(y_test, y_pred))

                   precision    recall  f1-score   support

              (A)       0.50      0.19      0.28        36
         (A),(Ad)       0.00      0.00      0.00         1
          (A),(H)       0.00      0.00      0.00         1
           (A),Ad       0.00      0.00      0.00         1
           (A),An       0.00      0.00      0.00         1
          (A),Art       0.00      0.00      0.00         3
           (A),Bl       0.00      0.00      0.00         2
           (A),Cg       0.00      0.00      0.00         2
           (A),Fi       0.00      0.00      0.00         3
           (A),Hx       0.00      0.00      0.00         1
        (A),Hx,Fi       0.00      0.00      0.00         1
           (A),Sc       0.00      0.00      0.00         1
             (Ad)       0.00      0.00      0.00        10
       (Ad),Ad,Fi       0.00      0.00      0.00         1
          (Ad),An       0.00      0.00      0.00         1
         (Ad),Art       0.00      0.00      0.00       

C:\Users\Aubin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Aubin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Aubin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: Undefi

#### TODO

reduce Content/label numbers